In [1]:
import numpy as np
from scipy.stats import unitary_group
from numpy import linalg as LA
from scipy.linalg import block_diag
from scipy.linalg import logm, expm
import qiskit

This program defines random states rho_i of two "distant" electrons occupying two orthogonal subspaces of dim. dA, dB in a 1P space of dim d, one electron per subspace; we compute the corresponding classical minimizer and quantum correlation (up to subtracting rho_i's entropy) treating the fermions as dist. particles and then as fermions. First minimization is over SU(dA) x SU(dA). Second minimization is over SU(d). Goal is to understand whether the minimizing 1P basis for the second, bigger minimization is "localized", i.e. made up of modes $a_j, b_j$. the question is then whether the two minimizations yield same result.


OLD, TO BE RENEWED:

The random matrix rho represents the random state wrt the basis

$|10 \; 10 \rangle, |10\;01 \rangle, |01\;10 \rangle, |01\;01 \rangle , | 11\;00\rangle, |00\;11 \rangle$

relative to some reference orbital $a_1, a_2, b_1, b_2$. The first four vectors span the "allowed" subspace of distant fermions, correponding to a 2-qbit space.

In [2]:
#parameters. (visible to functions as well, no need to pass them)

d=4
dA=2
dB=2

M=5 #number of states rho_i

mdist=2000000 #number of unitaries in global otpimization for dist particles
mpodium1dist=500 #unitaries on first podium after global sampling
mpodium2dist=200 #unitaries on second podium

mlocal1dist=500 #number of random steps
epsilon1dist=0.1  #length of random steps
mlocal2dist=300 #number of random steps
epsilon2dist=0.005  #length of random steps
mlocal3dist=20000 #number of random steps
epsilon3dist=0.001  #length of random steps
mlocal4dist=50000 #number of random steps
epsilon4dist=0.0005  #length of random steps
mlocal5dist=100000 #number of random steps
epsilon5dist=0.0001  #length of random steps

mferm=2000000 #number of unitaries in global otpimization for FERMIONS 
mpodium1ferm=500 #unitaries on first podium after global sampling
mpodium2ferm=200 #unitaries on second podium

mlocal1ferm=500 #number of random steps
epsilon1ferm=0.1  #length of random steps
mlocal2ferm=300 #number of random steps
epsilon2ferm=0.005  #length of random steps
mlocal3ferm=20000 #number of random steps
epsilon3ferm=0.001  #length of random steps
mlocal4ferm=5000 #number of random steps
epsilon4ferm=0.0005  #length of random steps
mlocal5ferm=100000 #number of random steps
epsilon5ferm=0.0001  #length of random steps

In [3]:
#basic functions

eps=1.e-13
def h(x):
    if x<eps:
        return(0)
    else:
        return(-x*np.log(x))

def shannon(distrib):
    summ=0.
    for j in range(len(distrib)):
        summ=summ+h(distrib[j])
    return(summ)

Tdist=[] #translates a global index into a couple of local indeces relative to usual lexicographic order
for j in range(dA):
    for k in range(dB):
        Tdist.append([j,k])

def inverseTdist(alpha,beta):#translates a couple of local indeces into a global index
    return(alpha*dB+beta)
        
def liftdist(uA,uB): #lifts uA, uB to uA tensor uB
    U=np.empty((dA*dB,dA*dB),dtype=complex)
    for j in range(dA*dB):
        for k in range(dA*dB):
            U[j][k]=uA[Tdist[j][0]][Tdist[k][0]]*uB[Tdist[j][1]][Tdist[k][1]]
    return(U)
    
def Fdist(rho,uA,uB): #takes a dA*dB x dA*dB* density matrix on HA tensor HB, two local rotatations
                        #returns optimizand number
    U=liftdist(uA,uB)
#    Udag=np.conjugate(U).T
    diag=[]
    for j in range(dA*dB):
        diag.append((np.matmul(U[j],np.matmul(rho,np.conjugate(U[j])))).real)
    return(shannon(diag))

In [4]:
#returns a dimxdim unitary epsilon-close to identity
def unitaryclosetoid(epsilon,dim):
    M = np.random.rand(dim, dim)-0.5 + 1j*np.random.rand(dim, dim)  # a (very basic) random complex matrix
    A = (M - np.conj(M.T)) #make it antihermitian. eigenvalues are like between -3i and 3i...
    deltaU=expm(epsilon*A) #unitary close to identity
    return(deltaU)

def newpodiumdist(oldpodium,mnewpodium,localm,eps): #mnewpodium = length of new podium; localm and eps = parameters for local maximization
    podium=np.copy(oldpodium) #so as not to change oldpodium, just in case
    deltauA=np.identity(dA,dtype=complex)
    deltauB=np.identity(dB,dtype=complex)
    uAtemp=np.identity(dA,dtype=complex)
    uBtemp=np.identity(dB,dtype=complex)    
    mpodium=len(podium[0])
    v=np.zeros(d)
    Ftemp=100.
    for k in range(localm):
        deltauA=unitaryclosetoid(eps,dA)
        deltauB=unitaryclosetoid(eps,dB)
        for i in range(M): #all unitaries treated independently
            for l in range(mpodium): #treat  podium 3tuples  (F value,uA,uB) independently
                uAtemp = np.matmul(podium[i][l][1],deltauA) #unitary close to podium[i][l][1]
                uBtemp = np.matmul(podium[i][l][2],deltauB)             #unitary close to podium[i][l][2]
                Ftemp = Fdist(rhos[i],uAtemp,uBtemp)
                if Ftemp<podium[i][l][0]:
                    podium[i][l][0]=Ftemp
                    podium[i][l][1]=np.copy(uAtemp)
                    podium[i][l][2]=np.copy(uBtemp) 
    for i in range(M):
        podium[i] = sorted(podium[i], key=lambda x: -x[0])
    newpodium=[[podium[i][-mnewpodium+j] for j in range(mnewpodium)] for i in range(M) ]#only keep the best mnewpodium guys
    return(newpodium)

In [5]:
#main variables

rhos=[]
for i in range(M):
#    rhos.append(qiskit.quantum_info.random_density_matrix(dA*dB))
    rhos.append(qiskit.quantum_info.random_density_matrix(dA*dB,1)) #for check with pure states
#Fdistvalues=[100.]*M  MAYBE NEEDED LATER #this list will be filled with Fdist minimum, one per rho
#Ffermvalues=[100.]*M   #this list will be filled with Fferm minimum, one per rho

In [6]:
#run optimization for distinguishable particles

#allocate memory for some relevant variables?
#Ubest=np.identity(d,dtype=complex)
#U=np.identity(d,dtype=complex)
podium1dist=[[[100.,np.identity(dA,dtype=complex),np.identity(dB,dtype=complex)] for j in range(mpodium1dist)]for i in range(M)]
#for each i=1,...,M podium1dist[i] will get filled with 3-tuples (F value, uA,uB)
podium2dist=[[[100.,np.identity(dA,dtype=complex),np.identity(dB,dtype=complex)] for j in range(mpodium2dist)]for i in range(M)]
podium3dist=[[[100.,np.identity(dA,dtype=complex),np.identity(dB,dtype=complex)]]for i in range(M)]
winnerdist=[[100.,np.identity(dA,dtype=complex),np.identity(dB,dtype=complex)]for i in range(M)] #remove nested brackets

#step 0: global optimization. altogether for all rhos_i!!
for k in range(mdist):
    uA=unitary_group.rvs(dA)
    uB=unitary_group.rvs(dB)
    for i in range(M):
        newvalue=Fdist(rhos[i],uA,uB)
        if newvalue<podium1dist[i][0][0]:
            podium1dist[i][0][0]=newvalue
            podium1dist[i][0][1]=np.copy(uA)
            podium1dist[i][0][2]=np.copy(uB)
            #order podium1; best (F ,uA,uB) i.e. with lowest F is given by podium1[i][-1]
            podium1dist[i] = sorted(podium1dist[i], key=lambda x: -x[0])
            
#step 1: shrink podium1dist to podium2dist

podium2dist=np.copy(newpodiumdist(podium1dist,mpodium2dist,mlocal1dist,epsilon1dist))

#step 2: shrink podium2dist to podium3dist
podium3dist=np.copy(newpodiumdist(podium2dist,1,mlocal2dist,epsilon2dist))

#step 3: optimize winner
podium3dist=np.copy(newpodiumdist(podium3dist,1,mlocal3dist,epsilon3dist))
podium3dist=np.copy(newpodiumdist(podium3dist,1,mlocal4dist,epsilon4dist))
podium3dist=np.copy(newpodiumdist(podium3dist,1,mlocal5dist,epsilon5dist))

#remove nested brackets
for i in range(M): #remove nested brackets
    winnerdist[i]=podium3dist[i][0]

/software/opt/focal/x86_64/python/3.10-2022.08/lib/python3.10/site-packages/numpy/lib/function_base.py:935: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


In [7]:
#the following checks that for PURE states you do get entropy of squared schmidt coefficients

def rhoA(rho): #rho must be a numpy nd array. if you want to input a  qiskit dens matrices, use .data to convert it
    rhoA=np.identity(dA,dtype=complex)
    for i in range(dA):
        for j in range(dA):
            summ=0.
            for l in range(dB):
                summ=summ+rho[inverseTdist(i,l)][inverseTdist(j,l)]
            rhoA[i][j]=summ
    return(rhoA)

def rhoAentropy(rhoA):
    return shannon(LA.eigvals(rhoA))

actualminima=[rhoAentropy(rhoA(rhos[i].data)) for i in range(M)]

#the following numbers should be all positive and very small
for i in range(M):
    print(winnerdist[i][0]-actualminima[i])

(0.0006819855857136403+1.0890758398608976e-16j)
(1.2958910194926787e-09+0j)
(0.0007443204274676374-2.8398024352793345e-17j)
(0.022896533785666306+1.0896675918184392e-17j)
(0.008396241760433937-5.463007277623801e-18j)


In [8]:
#unitarity check...

In [9]:
#to optimize further
podium3dist=np.copy(newpodiumdist(podium3dist,1,10000,0.00001))

#remove nested brackets
for i in range(M): #remove nested brackets
    winnerdist[i]=podium3dist[i][0]

In [10]:
#now treat fermions as fermions. I start with dA=dB=2,d=4 to check convergence etc. Looks already quite tricky...
# i choose basis for two fermion hilbert space where you have the four SDs of far apart fermions first
#and the other two ones next

dA=2
dB=2
d=4
D=6 #dim of 2 fermion hilbert space
rhosF = [block_diag(rhos[i],np.zeros((2,2),dtype=complex)) for i in range(M)]

#functions

#translation of indeces for fermions
Tferm=np.array([[0,2],[0,3],[1,2],[1,3],[0,1],[2,3]])

#lifts a 1P 4x4 unitary u to a 2P 6x6 unitary W
#W will be used to transform rhoF[i]'s components
def liftferm(u):
    W=np.zeros((D,D),dtype=complex)
    for m in range(D):
        for j in range(D):
            W[m][j]=u[Tferm[m][0]][Tferm[j][0]]*u[Tferm[m][1]][Tferm[j][1]]-u[Tferm[m][1]][Tferm[j][0]]*u[Tferm[m][0]][Tferm[j][1]]
    return W

def Fferm(rho,u): #takes a density matrix rho on 2 fermion space and a 1P rotatation u
                        #returns optimizand number
    U=liftferm(u)
#    Udag=np.conjugate(U).T
    diag=[]
    for j in range(D):
        diag.append((np.matmul(U[j],np.matmul(rho,np.conjugate(U[j])))).real)
    return(shannon(diag))

def newpodiumferm(oldpodium,mnewpodium,localm,eps): #mnewpodium = length of new podium; localm and eps = parameters for local maximization
    podium=np.copy(oldpodium) #so as not to change oldpodium, just in case
    deltau=np.identity(d,dtype=complex)
    utemp=np.identity(d,dtype=complex)  
    mpodium=len(podium[0])
    Ftemp=100.
    for k in range(localm):
        deltau=unitaryclosetoid(eps,d)
        for i in range(M): #all unitaries treated independently
            for l in range(mpodium): #treat  podium couples  (F value,u) independently
                utemp = np.matmul(podium[i][l][1],deltau) #unitary close to podium[i][l][1]
                Ftemp = Fferm(rhosF[i],utemp)
                if Ftemp<podium[i][l][0]:
                    podium[i][l][0]=Ftemp
                    podium[i][l][1]=np.copy(utemp)
    for i in range(M):
        podium[i] = sorted(podium[i], key=lambda x: -x[0])
    newpodium=[[podium[i][-mnewpodium+j] for j in range(mnewpodium)] for i in range(M) ]#only keep the best mnewpodium guys
    return(newpodium)

#run optimization for fermions

podium1ferm=[[[100.,np.identity(d,dtype=complex)] for j in range(mpodium1ferm)]for i in range(M)]
#for each i=1,...,M podium1ferm[i] will get filled with couples (F value, u)
podium2ferm=[[[100.,np.identity(d,dtype=complex)] for j in range(mpodium2ferm)]for i in range(M)]
podium3ferm=[[[100.,np.identity(d,dtype=complex)]]for i in range(M)]
winnerferm=[[100.,np.identity(d,dtype=complex)]for i in range(M)] #remove nested brackets

#step 0: global optimization. altogether for all rhosF_i!!
for k in range(mferm):
    u=unitary_group.rvs(d)
    for i in range(M):
        newvalue=Fferm(rhosF[i],u)
        if newvalue<podium1ferm[i][0][0]:
            podium1ferm[i][0][0]=newvalue
            podium1ferm[i][0][1]=np.copy(u)
            #order podia; best (F ,u) i.e. with lowest F is given by podium1[i][-1]
            podium1ferm[i] = sorted(podium1ferm[i], key=lambda x: -x[0])
            
#step 1: shrink podium1ferm to podium2ferm

podium2ferm=np.copy(newpodiumferm(podium1ferm,mpodium2ferm,mlocal1ferm,epsilon1ferm))

#step 2: shrink podium2ferm to podium3ferm
podium3ferm=np.copy(newpodiumferm(podium2ferm,1,mlocal2ferm,epsilon2ferm))

#step 3: optimize winner
podium3ferm=np.copy(newpodiumferm(podium3ferm,1,mlocal3ferm,epsilon3ferm))
podium3ferm=np.copy(newpodiumferm(podium3ferm,1,mlocal4ferm,epsilon4ferm))
podium3ferm=np.copy(newpodiumferm(podium3ferm,1,mlocal5ferm,epsilon5ferm))

#remove nested brackets
for i in range(M): #remove nested brackets
    winnerferm[i]=podium3ferm[i][0]

In [11]:
#comparison between the two minimizations (and analytical result for pure states):
print("fermions --- dist particles --- (analytical result, only if you chose pure states)")
for i in range(M):
    print(winnerferm[i][0],winnerdist[i][0],actualminima[i],"\n")

    
#the hope is that for mixed states the value obtained for dist particles and that obtained for fermions
#are the same, also for MIXED states. Ofc it does not make sense to check this if things are not converging.

fermions --- dist particles --- (analytical result, only if you chose pure states)
0.35605330022238374 0.07415897308063286 (0.07349108525004254-1.0890758398608976e-16j) 

0.25454394865996277 0.06857664834933891 (0.06857664818263984+0j) 

0.5824208468986518 0.2915691613146246 (0.29082498601661594+2.8398024352793345e-17j) 

0.4665697423661938 0.3507873484028735 (0.3279052916816982-1.0896675918184392e-17j) 

0.21885848047517165 0.1473829159741423 (0.13898742946479867+5.463007277623801e-18j) 



In [12]:
#PURE STATES